In [21]:
import numpy as np
import random
import pickle
from simple_custom_taxi_env import SimpleTaxiEnv
import time

In [22]:
np.bool8 = np.bool_

env_config = {
    "fuel_limit": 5000
}
render = False
hyperparameters = {
    "alpha": 0.001,
	"gamma": 0.99,
	"epsilon_start": 1.0, 
	"epsilon_end": 0.1,
	"decay_rate": 0.9995,
	"episodes": 5000,
 	"max_steps": 2000
}

In [ ]:
q_table = {}
epsilon = hyperparameters["epsilon_start"]

def get_state(obs, target_loc=None, has_picked_up=False):
	stations = [[0, 0], [0, 4], [4, 0], [4,4]]
	taxi_row, taxi_col, stations[0][0],stations[0][1] ,stations[1][0],stations[1][1],stations[2][0],stations[2][1],stations[3][0],stations[3][1],obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
	stations = [tuple(i) for i in stations]	

	assert target_loc is not None
	x_dir = target_loc[0] - taxi_row
	y_dir = target_loc[1] - taxi_col
	x_dir = 0 if x_dir == 0 else x_dir // abs(x_dir)
	y_dir = 0 if y_dir == 0 else y_dir // abs(y_dir)
	return (x_dir, y_dir, obstacle_north, obstacle_south, obstacle_east, obstacle_west, has_picked_up)
  
def get_action(obs):
	"""
	# Selects the best action using the trained Q-table.
	"""
	if np.random.uniform(0, 1) < epsilon:
		action = np.random.choice(action_nums)  # Random action
	else:
		action = np.argmax(q_table[get_state(obs)])  # Greedy action
	return action

def is_in_station(obs):
	"""
	# Checks if the taxi is in a station.
	"""
	stations = [[0, 0], [0, 4], [4, 0], [4,4]]
	taxi_row, taxi_col,stations[0][0],stations[0][1] ,stations[1][0],stations[1][1],stations[2][0],stations[2][1],stations[3][0],stations[3][1],obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
	stations = [tuple(i) for i in stations]
	return (taxi_row, taxi_col) in stations

env = SimpleTaxiEnv(**env_config)
action_nums = 6
rewards_per_episode = []

obs, _ = env.reset()
total_reward = 0
done = False
step_count = 0
stations = [(0, 0), (0, 4), (4, 0), (4,4)]

taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs

if render:
	env.render_env((taxi_row, taxi_col),
					action=None, step=step_count, fuel=env.current_fuel)
	time.sleep(0.5)
 

for episode in range(hyperparameters["episodes"]):
	get_state.passenger_loc, get_state.destination_loc = None, None
	obs, _ = env.reset()
	done = False
	total_reward = 0
	step_count = 0
 
	destination = None
	visited = []
	has_picked_up = False
 
	taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
	stations = [(obs[2], obs[3]), (obs[4], obs[5]), (obs[6], obs[7]), (obs[8], obs[9])]
	target_loc = stations[0]
	state = get_state(obs, target_loc, has_picked_up)
	
	while not done:	
		if state not in q_table:
			q_table[state] = np.zeros(action_nums)
   
		taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
		if np.random.uniform(0, 1) < epsilon:
			action_probs = np.ones(action_nums) / action_nums
			if obstacle_south:
				action_probs[0] = 0
			if obstacle_north:
				action_probs[1] = 0
			if obstacle_east:
				action_probs[2] = 0
			if obstacle_west:
				action_probs[3] = 0	
			if not passenger_look or has_picked_up or not is_in_station(obs):
				action_probs[4] = 0
			if not destination_look or not has_picked_up or not is_in_station(obs):
				action_probs[5] = 0
			action_probs = action_probs / np.sum(action_probs)
			action = np.random.choice(action_nums, p=action_probs)  # Random action
		else:
			action = np.argmax(q_table[state])  # Greedy action
		
		if not has_picked_up and passenger_look and is_in_station(obs) and action == 4:
			has_picked_up = True
		if has_picked_up and destination_look and is_in_station(obs) and action == 5:
			done = True
   
		shaped_reward = 0
		x_dir = target_loc[0] - taxi_row
		y_dir = target_loc[1] - taxi_col
		x_dir = 0 if x_dir == 0 else x_dir // abs(x_dir)
		y_dir = 0 if y_dir == 0 else y_dir // abs(y_dir)
		# if x_dir == 1 and action == 1 and not obstacle_north:
		# 	shaped_reward = 1000
		# if x_dir == -1 and action == 0 and not obstacle_south:
		# 	shaped_reward = 1000
		# if y_dir == 1 and action == 3 and not obstacle_west:
		# 	shaped_reward = 1000
		# if y_dir == -1 and action == 2 and not obstacle_east:
		# 	shaped_reward = 1000
		if action == 4 and passenger_look and not has_picked_up and is_in_station(obs): 
			shaped_reward = 5000
		if action == 5 and destination_look and has_picked_up and is_in_station(obs):
			shaped_reward = 10000
		if obstacle_south and action == 0:
			shaped_reward = -100
			# print ("south")
		if obstacle_north and action == 1:
			shaped_reward = -100
			# print ("north")
		if obstacle_east and action == 2:
			shaped_reward = -100
			# print ("east")
		if obstacle_west and action == 3:
			shaped_reward = -100
			# print ("west")
		if (not passenger_look or has_picked_up or not is_in_station(obs)) and action == 4:
			shaped_reward = -100
			# print ("not passenger")
		if (not destination_look or not has_picked_up or not is_in_station(obs)) and action == 5:
			shaped_reward = -100
			# print ("not destination")
   
		next_obs, reward, done, _ = env.step(action)
		taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = next_obs
  
		if is_in_station(next_obs) and (taxi_row, taxi_col) not in visited:
			visited.append((taxi_row, taxi_col))
			if destination_look:
				destination = (taxi_row, taxi_col)
			if has_picked_up and destination is not None:
				target_loc = destination
			else:
				# choose a station that has not been visited yet
				# print (visited, (taxi_row, taxi_col))
				for station in stations:
					if station not in visited:
						target_loc = station
						break	
				# print (target_loc)

		# if done:
		# 	if not has_picked_up:
		# 		# shaped_reward = -100
		# 		# print ("not picked up", next_obs, action, has_picked_up, destination)
		# 		done = False
		# 	if has_picked_up and not destination_look:
		# 		# shaped_reward = -100
		# 		# print ("not destination")
		# 		done = False
		# 	if has_picked_up and destination_look:
		# 		# print ("done")
		# 		shaped_reward = 2000
		
		total_reward += reward
		reward += shaped_reward
		next_state = get_state(next_obs, target_loc, has_picked_up)
		if next_obs not in q_table:
			q_table[next_state] = np.zeros(action_nums)
		q_table[state][action] += hyperparameters["alpha"] * (reward + hyperparameters["gamma"] * np.max(q_table[next_state]) - q_table[state][action])
		
		step_count += 1
		obs = next_obs
		state = next_state
   
	
		if render:
			env.render_env((taxi_row, taxi_col),
							action=action, step=step_count, fuel=env.current_fuel)
	# print (step_count)
	rewards_per_episode.append(total_reward)
	epsilon = max(hyperparameters["epsilon_end"], epsilon * hyperparameters["decay_rate"])
	if (episode + 1) % 20 == 0:
		avg_reward = np.mean(rewards_per_episode[-20:])
		print(f'Episode {episode + 1}/{hyperparameters["episodes"]}, Avg Reward: {avg_reward:.4f}, Epsilon: {epsilon:.3f}')
		# print ([np.argmax(i) for i in q_table.values()])
		print (len(q_table))
  
filename = "q_table.pkl"

# 儲存 Q-table
with open(filename, "wb") as f:
    pickle.dump(q_table, f)

print(f"Q-table 已儲存至 {filename}")

Episode 20/5000, Avg Reward: 10.5600, Epsilon: 0.990
71
Episode 40/5000, Avg Reward: 0.1000, Epsilon: 0.980
72
Episode 60/5000, Avg Reward: 4.1400, Epsilon: 0.970
72
Episode 80/5000, Avg Reward: 1.9400, Epsilon: 0.961
72
Episode 100/5000, Avg Reward: 0.7350, Epsilon: 0.951
72
Episode 120/5000, Avg Reward: -9.4000, Epsilon: 0.942
72
Episode 140/5000, Avg Reward: -6.0050, Epsilon: 0.932
72
Episode 160/5000, Avg Reward: -20.6750, Epsilon: 0.923
72
Episode 180/5000, Avg Reward: -18.3500, Epsilon: 0.914
72
Episode 200/5000, Avg Reward: -35.6250, Epsilon: 0.905
72
Episode 220/5000, Avg Reward: -21.5150, Epsilon: 0.896
72
Episode 240/5000, Avg Reward: -15.3900, Epsilon: 0.887
72
Episode 260/5000, Avg Reward: -21.5600, Epsilon: 0.878
72
Episode 280/5000, Avg Reward: -45.6100, Epsilon: 0.869
72
Episode 300/5000, Avg Reward: -57.1000, Epsilon: 0.861
72
Episode 320/5000, Avg Reward: -70.3500, Epsilon: 0.852
72
Episode 340/5000, Avg Reward: -66.7300, Epsilon: 0.844
72
Episode 360/5000, Avg Reward:

KeyboardInterrupt: 